In [1]:
import cv2
import numpy as np
import math
import torch

In [12]:
cap = cv2.VideoCapture('D:/AmirKabir/tez/ewap_dataset/ewap_dataset/seq_hotel/seq_hotel.avi')
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
total_frames

19350

In [6]:
import cv2
import numpy as np

def video_to_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    # Loop through all frames
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
        else:
            break
    # Release the video capture object
    cap.release()
    frames = np.array(frames)

    return frames

In [ ]:
video_path = 'D:/AmirKabir/tez/eighen trajectory/ewap_dataset/ewap_dataset/seq_hotel/seq_hotel.avi'
frame = video_to_frames(video_path)

In [3]:
import cv2
# Open the video file
cap = cv2.VideoCapture('D:/AmirKabir/tez/eighen trajectory/ewap_dataset/ewap_dataset/seq_hotel/seq_hotel.avi')
# Set the frame position to 10
cap.set(cv2.CAP_PROP_POS_FRAMES, 19260)

# Read the frame at position 10
ret, frame = cap.read()

# Check if the frame is valid
if ret:
    # Display the frame
    cv2.imshow('Frame 11', frame)
    cv2.waitKey(0)
else:
    print("Error reading frame")

# Release the video capture object
cap.release()
cv2.destroyAllWindows()

In [4]:
frame.shape

(576, 720, 3)

In [4]:
import cv2
import numpy as np
from torchvision.transforms import transforms
from PIL import Image

def prepare_image(frame):
    transform = transforms.Compose([
                      transforms.ToPILImage(),
                      transforms.Resize((300, 300))
                      ])
    transformed_image = transform(frame)
    return transformed_image

def video_to_frames(video_path, output_dir):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_num = 0
    # Loop through all frames
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            image = prepare_image(frame)
            image.save(f"{output_dir}/frame_{frame_num}.jpg")
            frame_num += 10
        else:
            break
    # Release the video capture object
    cap.release()

In [6]:
video_path = 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_test_cpu/datasets/eth/train/crowds_zara01.avi'
output_dir = 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_test_cpu/datasets/eth/train/crowds_zara01_frames'
video_to_frames(video_path,output_dir)
video_path = 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_test_cpu/datasets/eth/train/crowds_zara02.avi'
output_dir = 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_test_cpu/datasets/eth/train/crowds_zara02_frames'
video_to_frames(video_path,output_dir)
video_path = 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_test_cpu/datasets/eth/train/students003.avi'
output_dir = 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_test_cpu/datasets/eth/train/students003_frames'
video_to_frames(video_path,output_dir)

In [1]:
import os
import math
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data.sampler import Sampler
from torch.utils.data.dataloader import DataLoader
from torchvision.transforms import transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def get_dataloader(data_dir, phase, obs_len, pred_len, batch_size):
    r"""Get dataloader for a specific phase

    Args:
        data_dir (str): path to the dataset directory
        phase (str): phase of the data, one of 'train', 'val', 'test'
        obs_len (int): length of observed trajectory
        pred_len (int): length of predicted trajectory
        batch_size (int): batch size

    Returns:
        loader_phase (torch.utils.data.DataLoader): dataloader for the specific phase
    """

    assert phase in ['train', 'val', 'test']
    
    
    data_set = data_dir + '/' + phase + '/'
    shuffle = True if phase == 'train' else False
    drop_last = True if phase == 'train' else False 
    dataset_phase = TrajectoryDataset(data_set, obs_len=obs_len, pred_len=pred_len)
    sampler_phase = None
    if batch_size > 1:
        sampler_phase = TrajBatchSampler(dataset_phase, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    loader_phase = DataLoader(dataset_phase, batch_sampler=sampler_phase ,collate_fn=traj_collate_fn, pin_memory=True)
    return loader_phase


def traj_collate_fn(data):
    r"""Collate function for the dataloader

    Args:
        data (list): list of tuples of (obs_seq, pred_seq, non_linear_ped, loss_mask, seq_start_end)

    Returns:
        obs_seq_list (torch.Tensor): (num_ped, obs_len, 2)
        pred_seq_list (torch.Tensor): (num_ped, pred_len, 2)
        non_linear_ped_list (torch.Tensor): (num_ped,)
        loss_mask_list (torch.Tensor): (num_ped, obs_len + pred_len)
        scene_mask (torch.Tensor): (num_ped, num_ped)
        seq_start_end (torch.Tensor): (num_ped, 2)
        frame_list_tensor
    """

    obs_seq_list, pred_seq_list, non_linear_ped_list, loss_mask_list, _,frame_list_tensor= zip(*data)

    _len = [len(seq) for seq in obs_seq_list]
    cum_start_idx = [0] + np.cumsum(_len).tolist()
    seq_start_end = [[start, end] for start, end in zip(cum_start_idx, cum_start_idx[1:])]
    seq_start_end = torch.LongTensor(seq_start_end)
    scene_mask = torch.zeros(sum(_len), sum(_len), dtype=torch.bool)
    for idx, (start, end) in enumerate(seq_start_end):
        scene_mask[start:end, start:end] = 1

    out = [torch.cat(obs_seq_list, dim=0), torch.cat(pred_seq_list, dim=0),
           torch.cat(non_linear_ped_list, dim=0), torch.cat(loss_mask_list, dim=0), scene_mask, seq_start_end,frame_list_tensor]
    return tuple(out)


class TrajBatchSampler(Sampler):
    r"""Samples batched elements by yielding a mini-batch of indices.
    Args:
        data_source (Dataset): dataset to sample from
        batch_size (int): Size of mini-batch.
        shuffle (bool, optional): set to ``True`` to have the data reshuffled
            at every epoch (default: ``False``).
        drop_last (bool): If ``True``, the sampler will drop the last batch if
            its size would be less than ``batch_size``
        generator (Generator): Generator used in sampling.
    """

    def __init__(self, data_source, batch_size=64, shuffle=False, drop_last=False, generator=None):
        self.data_source = data_source
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.drop_last = drop_last
        self.generator = generator

    def __iter__(self):
        assert len(self.data_source) == len(self.data_source.num_peds_in_seq)

        if self.shuffle:
            if self.generator is None:
                generator = torch.Generator()
                generator.manual_seed(int(torch.empty((), dtype=torch.int64).random_().item()))
            else:
                generator = self.generator
            indices = torch.randperm(len(self.data_source), generator=generator).tolist()
        else:
            indices = list(range(len(self.data_source)))
        num_peds_indices = self.data_source.num_peds_in_seq[indices]

        batch = []
        total_num_peds = 0
        for idx, num_peds in zip(indices, num_peds_indices):
            batch.append(idx)
            total_num_peds += num_peds
            if total_num_peds >= self.batch_size:
                yield batch
                batch = []
                total_num_peds = 0
        if len(batch) > 0 and not self.drop_last:
            yield batch

    def __len__(self):
        # Approximated number of batches.
        # The order of trajectories can be shuffled, so this number can vary from run to run.
        if self.drop_last:
            return sum(self.data_source.num_peds_in_seq) // self.batch_size
        else:
            return (sum(self.data_source.num_peds_in_seq) + self.batch_size - 1) // self.batch_size


def read_file(_path, delim='\t'):
    data = []
    if delim == 'tab':
        delim = '\t'
    elif delim == 'space':
        delim = ' '
    with open(_path, 'r') as f:
        for line in f:
            line = line.strip().split(delim)
            line = [float(i) for i in line]
            data.append(line)
    return np.asarray(data)


def poly_fit(traj, traj_len, threshold):
    """
    Input:
    - traj: Numpy array of shape (2, traj_len)
    - traj_len: Len of trajectory
    - threshold: Minimum error to be considered for non-linear traj
    Output:
    - int: 1 -> Non Linear 0-> Linear
    """
    t = np.linspace(0, traj_len - 1, traj_len)
    res_x = np.polyfit(t, traj[0, -traj_len:], 2, full=True)[1]
    res_y = np.polyfit(t, traj[1, -traj_len:], 2, full=True)[1]
    if res_x + res_y >= threshold:
        return 1.0
    else:
        return 0.0

def prepare_image(frame):
    transform = transforms.Compose([
                      transforms.ToPILImage(),
                      transforms.Resize(300),
                      transforms.ToTensor()])
    transformed_image = transform(frame)
    transformed_image = transformed_image.unsqueeze(0)
    return transformed_image.to(device)



# In your TrajectoryDataset class



class TrajectoryDataset(Dataset):
    """Dataloder for the Trajectory datasets"""

    def __init__(self, data_dir, obs_len=8, pred_len=12, skip=1, threshold=0.02, min_ped=1, delim='\t'):
        """
        Args:
        - data_dir: Directory containing dataset files in the format <frame_id> <ped_id> <x> <y>
        - obs_len: Number of time-steps in input trajectories
        - pred_len: Number of time-steps in output trajectories
        - skip: Number of frames to skip while making the dataset
        - threshold: Minimum error to be considered for non-linear traj when using a linear predictor
        - min_ped: Minimum number of pedestrians that should be in a sequence
        - delim: Delimiter in the dataset files
        """
        super(TrajectoryDataset, self).__init__()

        self.data_dir = data_dir
        self.obs_len = obs_len
        self.pred_len = pred_len
        self.skip = skip
        self.seq_len = self.obs_len + self.pred_len
        self.delim = delim

        all_file = os.listdir(self.data_dir)
        all_file = [os.path.join(self.data_dir, _path) for _path in all_file]
        num_peds_in_seq = []
        seq_list = []
        frame_list = []
        loss_mask_list = []
        non_linear_ped = []
        pedestrain_id = []#new add
        text_format = 'txt'

        for name in all_file:
            path = None
            frame_path = None
            if name[-3:] == text_format:
                path = name 
                frame_path = name[:-4] + '_frames/'
 
                try:
                    data = read_file(path, delim)
                    video_path = frame_path
                except:
                    data = read_file(path, delim = 'space')    
                    video_path = frame_path
                    
                frame_id = [] #new add    
                frames = np.unique(data[:, 0]).tolist()
                frame_data = []
                for frame in frames:
                    frame_data.append(data[frame == data[:, 0], :])
                num_sequences = int(math.ceil((len(frames) - self.seq_len + 1) / skip))

                for idx in range(0, num_sequences * self.skip + 1, skip):
                    curr_seq_data = np.concatenate(frame_data[idx:idx + self.seq_len], axis=0)
                    frame_range = np.unique(curr_seq_data[:, 0])       #new add
                    peds_in_curr_seq = np.unique(curr_seq_data[:, 1])
                    curr_seq = np.zeros((len(peds_in_curr_seq), 2, self.seq_len))
                    curr_loss_mask = np.zeros((len(peds_in_curr_seq), self.seq_len))
                    num_peds_considered = 0
                    _non_linear_ped = []
                    ped_list = [] #new add
                    for _, ped_id in enumerate(peds_in_curr_seq):
                        curr_ped_seq = curr_seq_data[curr_seq_data[:, 1] == ped_id, :]
                        curr_ped_seq = np.around(curr_ped_seq, decimals=4)
                        pad_front = frames.index(curr_ped_seq[0, 0]) - idx
                        pad_end = frames.index(curr_ped_seq[-1, 0]) - idx + 1
                        if pad_end - pad_front != self.seq_len:
                            continue
                        curr_ped_seq = np.transpose(curr_ped_seq[:, 2:])
                        curr_ped_seq = curr_ped_seq
                        _idx = num_peds_considered
                        curr_seq[_idx, :, pad_front:pad_end] = curr_ped_seq
                        # Linear vs Non-Linear Trajectory
                        _non_linear_ped.append(poly_fit(curr_ped_seq, pred_len, threshold))
                        curr_loss_mask[_idx, pad_front:pad_end] = 1
                        num_peds_considered += 1
                        ped_list.append(ped_id) #new add

                    if num_peds_considered > min_ped:
                        non_linear_ped += _non_linear_ped
                        num_peds_in_seq.append(num_peds_considered)
                        loss_mask_list.append(curr_loss_mask[:num_peds_considered])
                        seq_list.append(curr_seq[:num_peds_considered])
                        frame_path = [video_path + 'frame_' + str(int(number)) + '.jpg' for number in frame_range][:8]
                        temp = [plt.imread(path) for path in frame_path]
                        frame_list.append(temp)
                        frame_id.append(frame_range) #new add
                        pedestrain_id.append(ped_list) #new add
                    
            
            # cap = cv2.VideoCapture(self.frame_path + '/seq_hotel.avi')
            # frame_lists = [(row.min() , row.max())  for row in frame_id] #new add
            # images_list = []
            # for tup in frame_lists:
            #     temp_list = []
            #     current_frame_number = tup[0]
            #     for i in range(obs_len):
            #         cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame_number)
            #         ret, current_frame = cap.read()
            #         frame = prepare_image(current_frame).to(device)
            #         temp_list.append(frame)
            #         current_frame_number+=10
            #     images_list.append(temp_list)
            # import pickle

            # Specify the full path to save the images_list as a pickle file
            # save_path = 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_test_cpu/datasets/image_List_hotel/images_list.pkl'
            # with open(save_path, 'wb') as f:
            #     pickle.dump(images_list, f)
            
        # with open(save_path, 'rb') as f:
        # self.frame_lists = [(row.min(), row.max()) for row in frame_id]
        # self.images = extract_frames(video_path, frame_lists, self.obs_len)
        self.num_seq = len(seq_list)
        seq_list = np.concatenate(seq_list, axis=0)
        # frame_list = np.concatenate(frame_list, axis=0)
        loss_mask_list = np.concatenate(loss_mask_list, axis=0)
        non_linear_ped = np.asarray(non_linear_ped)
         #new add
        self.pedestrain_id = np.concatenate(pedestrain_id , axis = 0) #new add
        self.num_peds_in_seq = np.array(num_peds_in_seq) 

        # Convert numpy -> Torch Tensor
        self.frame_list = frame_list
        self.obs_traj = torch.from_numpy(seq_list[:, :, :self.obs_len]).type(torch.float).permute(0, 2, 1)  # NTC
        self.pred_traj = torch.from_numpy(seq_list[:, :, self.obs_len:]).type(torch.float).permute(0, 2, 1)  # NTC
        self.loss_mask = torch.from_numpy(loss_mask_list).type(torch.float)
        self.non_linear_ped = torch.from_numpy(non_linear_ped).type(torch.float)
        cum_start_idx = [0] + np.cumsum(num_peds_in_seq).tolist()
        self.seq_start_end = [(start, end) for start, end in zip(cum_start_idx, cum_start_idx[1:])]

    def __len__(self):
        return self.num_seq
    
    # def extract_frames(self, video_path, min_frame, max_frame, obs_len):
    #     images_list = []
    #     cap =  cv2.VideoCapture(video_path) 
    #     temp_list = []
    #     current_frame_number = min_frame
    #     for _ in range(obs_len):
    #         cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame_number)
    #         ret, current_frame = cap.read()
    #         if not ret:
    #             raise RuntimeError(f"Could not read frame {current_frame_number} from {video_path}")
    #         frame = prepare_image(current_frame).to(device)
    #         temp_list.append(frame)
    #         current_frame_number += 10
    #     images_list.append(temp_list)
    #     return images_list

    def __getitem__(self, index):
        start, end = self.seq_start_end[index]
        # min_frame, max_frame= self.frame_lists[index]
        # frames = self.extract_frames(self.video_path, min_frame, max_frame, self.obs_len)
        out = [self.obs_traj[start:end], self.pred_traj[start:end],
               self.non_linear_ped[start:end], self.loss_mask[start:end], [[0, end - start]], self.frame_list[index]]
        return out 


In [1]:
import os
import math
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data.sampler import Sampler
from torch.utils.data.dataloader import DataLoader
from torchvision.transforms import transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def get_dataloader(data_dir, phase, obs_len, pred_len, batch_size):
    r"""Get dataloader for a specific phase

    Args:
        data_dir (str): path to the dataset directory
        phase (str): phase of the data, one of 'train', 'val', 'test'
        obs_len (int): length of observed trajectory
        pred_len (int): length of predicted trajectory
        batch_size (int): batch size

    Returns:
        loader_phase (torch.utils.data.DataLoader): dataloader for the specific phase
    """

    assert phase in ['train', 'val', 'test']
    
    
    data_set = data_dir + '/' + phase + '/'
    shuffle = True if phase == 'train' else False
    drop_last = True if phase == 'train' else False 
    dataset_phase = TrajectoryDataset(data_set, obs_len=obs_len, pred_len=pred_len)
    sampler_phase = None
    if batch_size > 1:
        sampler_phase = TrajBatchSampler(dataset_phase, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    loader_phase = DataLoader(dataset_phase, batch_sampler=sampler_phase ,collate_fn=traj_collate_fn, pin_memory=True)
    return loader_phase


def traj_collate_fn(data):
    r"""Collate function for the dataloader

    Args:
        data (list): list of tuples of (obs_seq, pred_seq, non_linear_ped, loss_mask, seq_start_end)

    Returns:
        obs_seq_list (torch.Tensor): (num_ped, obs_len, 2)
        pred_seq_list (torch.Tensor): (num_ped, pred_len, 2)
        non_linear_ped_list (torch.Tensor): (num_ped,)
        loss_mask_list (torch.Tensor): (num_ped, obs_len + pred_len)
        scene_mask (torch.Tensor): (num_ped, num_ped)
        seq_start_end (torch.Tensor): (num_ped, 2)
        frame_list_tensor
    """

    obs_seq_list, pred_seq_list, non_linear_ped_list, loss_mask_list, _,frame_list_tensor= zip(*data)

    _len = [len(seq) for seq in obs_seq_list]
    cum_start_idx = [0] + np.cumsum(_len).tolist()
    seq_start_end = [[start, end] for start, end in zip(cum_start_idx, cum_start_idx[1:])]
    seq_start_end = torch.LongTensor(seq_start_end)
    scene_mask = torch.zeros(sum(_len), sum(_len), dtype=torch.bool)
    for idx, (start, end) in enumerate(seq_start_end):
        scene_mask[start:end, start:end] = 1

    out = [torch.cat(obs_seq_list, dim=0), torch.cat(pred_seq_list, dim=0),
           torch.cat(non_linear_ped_list, dim=0), torch.cat(loss_mask_list, dim=0), scene_mask, seq_start_end,frame_list_tensor]
    return tuple(out)


class TrajBatchSampler(Sampler):
    r"""Samples batched elements by yielding a mini-batch of indices.
    Args:
        data_source (Dataset): dataset to sample from
        batch_size (int): Size of mini-batch.
        shuffle (bool, optional): set to ``True`` to have the data reshuffled
            at every epoch (default: ``False``).
        drop_last (bool): If ``True``, the sampler will drop the last batch if
            its size would be less than ``batch_size``
        generator (Generator): Generator used in sampling.
    """

    def __init__(self, data_source, batch_size=64, shuffle=False, drop_last=False, generator=None):
        self.data_source = data_source
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.drop_last = drop_last
        self.generator = generator

    def __iter__(self):
        assert len(self.data_source) == len(self.data_source.num_peds_in_seq)

        if self.shuffle:
            if self.generator is None:
                generator = torch.Generator()
                generator.manual_seed(int(torch.empty((), dtype=torch.int64).random_().item()))
            else:
                generator = self.generator
            indices = torch.randperm(len(self.data_source), generator=generator).tolist()
        else:
            indices = list(range(len(self.data_source)))
        num_peds_indices = self.data_source.num_peds_in_seq[indices]

        batch = []
        total_num_peds = 0
        for idx, num_peds in zip(indices, num_peds_indices):
            batch.append(idx)
            total_num_peds += num_peds
            if total_num_peds >= self.batch_size:
                yield batch
                batch = []
                total_num_peds = 0
        if len(batch) > 0 and not self.drop_last:
            yield batch

    def __len__(self):
        # Approximated number of batches.
        # The order of trajectories can be shuffled, so this number can vary from run to run.
        if self.drop_last:
            return sum(self.data_source.num_peds_in_seq) // self.batch_size
        else:
            return (sum(self.data_source.num_peds_in_seq) + self.batch_size - 1) // self.batch_size


def read_file(_path, delim='\t'):
    data = []
    if delim == 'tab':
        delim = '\t'
    elif delim == 'space':
        delim = ' '
    with open(_path, 'r') as f:
        for line in f:
            line = line.strip().split(delim)
            line = [float(i) for i in line]
            data.append(line)
    return np.asarray(data)


def poly_fit(traj, traj_len, threshold):
    """
    Input:
    - traj: Numpy array of shape (2, traj_len)
    - traj_len: Len of trajectory
    - threshold: Minimum error to be considered for non-linear traj
    Output:
    - int: 1 -> Non Linear 0-> Linear
    """
    t = np.linspace(0, traj_len - 1, traj_len)
    res_x = np.polyfit(t, traj[0, -traj_len:], 2, full=True)[1]
    res_y = np.polyfit(t, traj[1, -traj_len:], 2, full=True)[1]
    if res_x + res_y >= threshold:
        return 1.0
    else:
        return 0.0

def prepare_image(frame):
    transform = transforms.Compose([
                      transforms.ToPILImage(),
                      transforms.Resize(300),
                      transforms.ToTensor()])
    transformed_image = transform(frame)
    transformed_image = transformed_image.unsqueeze(0)
    return transformed_image.to(device)



class TrajectoryDataset(Dataset):
    """Dataloder for the Trajectory datasets"""

    def __init__(self, data_dir, obs_len=8, pred_len=12, skip=1, threshold=0.02, min_ped=1, delim='\t'):
        """
        Args:
        - data_dir: Directory containing dataset files in the format <frame_id> <ped_id> <x> <y>
        - obs_len: Number of time-steps in input trajectories
        - pred_len: Number of time-steps in output trajectories
        - skip: Number of frames to skip while making the dataset
        - threshold: Minimum error to be considered for non-linear traj when using a linear predictor
        - min_ped: Minimum number of pedestrians that should be in a sequence
        - delim: Delimiter in the dataset files
        """
        super(TrajectoryDataset, self).__init__()

        self.data_dir = data_dir
        self.obs_len = obs_len
        self.pred_len = pred_len
        self.skip = skip
        self.seq_len = self.obs_len + self.pred_len
        self.delim = delim

        all_file = os.listdir(self.data_dir)
        all_file = [os.path.join(self.data_dir, _path) for _path in all_file]
        num_peds_in_seq = []
        seq_list = []
        frame_list = []
        loss_mask_list = []
        non_linear_ped = []
        pedestrain_id = []#new add
        text_format = 'txt'

        for name in all_file:
            path = None
            frame_path = None
            if name[-3:] == text_format:
                path = name 
                frame_path = name[:-4] + '_frames/'
 
                try:
                    data = read_file(path, delim)
                    video_path = frame_path
                except:
                    data = read_file(path, delim = 'space')    
                    video_path = frame_path
                    
                frame_id = [] #new add    
                frames = np.unique(data[:, 0]).tolist()
                frame_data = []
                for frame in frames:
                    frame_data.append(data[frame == data[:, 0], :])
                num_sequences = int(math.ceil((len(frames) - self.seq_len + 1) / skip))

                for idx in range(0, num_sequences * self.skip + 1, skip):
                    curr_seq_data = np.concatenate(frame_data[idx:idx + self.seq_len], axis=0)
                    frame_range = np.unique(curr_seq_data[:, 0])       #new add
                    peds_in_curr_seq = np.unique(curr_seq_data[:, 1])
                    curr_seq = np.zeros((len(peds_in_curr_seq), 2, self.seq_len))
                    curr_loss_mask = np.zeros((len(peds_in_curr_seq), self.seq_len))
                    num_peds_considered = 0
                    _non_linear_ped = []
                    ped_list = [] #new add
                    for _, ped_id in enumerate(peds_in_curr_seq):
                        curr_ped_seq = curr_seq_data[curr_seq_data[:, 1] == ped_id, :]
                        curr_ped_seq = np.around(curr_ped_seq, decimals=4)
                        pad_front = frames.index(curr_ped_seq[0, 0]) - idx
                        pad_end = frames.index(curr_ped_seq[-1, 0]) - idx + 1
                        if pad_end - pad_front != self.seq_len:
                            continue
                        curr_ped_seq = np.transpose(curr_ped_seq[:, 2:])
                        curr_ped_seq = curr_ped_seq
                        _idx = num_peds_considered
                        curr_seq[_idx, :, pad_front:pad_end] = curr_ped_seq
                        # Linear vs Non-Linear Trajectory
                        _non_linear_ped.append(poly_fit(curr_ped_seq, pred_len, threshold))
                        curr_loss_mask[_idx, pad_front:pad_end] = 1
                        num_peds_considered += 1
                        ped_list.append(ped_id) #new add

                    if num_peds_considered > min_ped:
                        non_linear_ped += _non_linear_ped
                        num_peds_in_seq.append(num_peds_considered)
                        loss_mask_list.append(curr_loss_mask[:num_peds_considered])
                        seq_list.append(curr_seq[:num_peds_considered])
                        frame_path = [video_path + 'frame_' + str(int(number)) + '.jpg' for number in frame_range][:8]
                        temp = [plt.imread(path) for path in frame_path]
                        frame_list.append(temp)
                        frame_id.append(frame_range) #new add
                        pedestrain_id.append(ped_list) #new add
                    
            
        self.num_seq = len(seq_list)
        seq_list = np.concatenate(seq_list, axis=0)
        loss_mask_list = np.concatenate(loss_mask_list, axis=0)
        non_linear_ped = np.asarray(non_linear_ped)
         #new add
        self.pedestrain_id = np.concatenate(pedestrain_id , axis = 0) #new add
        self.num_peds_in_seq = np.array(num_peds_in_seq) 

        # Convert numpy -> Torch Tensor
        self.frame_list = frame_list
        self.obs_traj = torch.from_numpy(seq_list[:, :, :self.obs_len]).type(torch.float).permute(0, 2, 1)  # NTC
        self.pred_traj = torch.from_numpy(seq_list[:, :, self.obs_len:]).type(torch.float).permute(0, 2, 1)  # NTC
        self.loss_mask = torch.from_numpy(loss_mask_list).type(torch.float)
        self.non_linear_ped = torch.from_numpy(non_linear_ped).type(torch.float)
        cum_start_idx = [0] + np.cumsum(num_peds_in_seq).tolist()
        self.seq_start_end = [(start, end) for start, end in zip(cum_start_idx, cum_start_idx[1:])]

    def __len__(self):
        return self.num_seq


    def __getitem__(self, index):
        start, end = self.seq_start_end[index]
        out = [self.obs_traj[start:end], self.pred_traj[start:end],
               self.non_linear_ped[start:end], self.loss_mask[start:end], [[0, end - start]], self.frame_list[index]]
        return out 


In [2]:
dataset_dir = 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_test_cpu/datasets/eth/train/'
all_file = os.listdir(dataset_dir)
# all_files = [p for p in all_file if p[-3:] == text_format]
all_file = [os.path.join(dataset_dir, _path) for _path in all_file]
all_file

['D:/AmirKabir/tez/eighen trajectory/project_git/project/project_test_cpu/datasets/eth/train/biwi_hotel_train.txt',
 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_test_cpu/datasets/eth/train/biwi_hotel_train_frames',
 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_test_cpu/datasets/eth/train/crowds_zara01.txt',
 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_test_cpu/datasets/eth/train/crowds_zara01_frames',
 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_test_cpu/datasets/eth/train/crowds_zara02.txt',
 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_test_cpu/datasets/eth/train/crowds_zara02_frames',
 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_test_cpu/datasets/eth/train/students003.txt',
 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_test_cpu/datasets/eth/train/students003_frames']

In [2]:
dataset_dir = 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_test_cpu/datasets/eth/'
loader_train = get_dataloader(dataset_dir, 'train', 8, 12 ,batch_size=1)

In [ ]:
for cnt, batch in enumerate(loader_train, desc=f'Train Epoch {epoch}', mininterval=1):
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            obs_traj, pred_traj = [tensor.to(device, non_blocking=True) for tensor in batch[:2]]
            frames = [[tensor.to(device, non_blocking=True) for tensor in sublist] for sublist in batch[-1][0]]

In [3]:
data = iter(loader_train)

In [4]:
test = next(data)

In [12]:
test[-1][0][0].shape

(300, 300, 3)

In [15]:
frames = [torch.from_numpy(np.copy(sublist)).to(device, non_blocking=True) for sublist in test[-1][0]]

In [5]:
test[6][0][0].shape

(300, 300, 3)

In [ ]:
all_data = []
for data  in loader_train:
    all_data.append(data)

In [ ]:
len(all_data)

In [3]:
for batch in loader_train:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    obs_traj, pred_traj = [tensor.to(device, non_blocking=True) for tensor in batch[:2]]
    frames = [[tensor.to(device, non_blocking=True) for tensor in sublist] for sublist in batch[-1][0]]

In [8]:
len(frames[0])

8

In [2]:
def read_file(_path, delim='\t'):
    data = []
    if delim == 'tab':
        delim = '\t'
    elif delim == 'space':
        delim = ' '
    with open(_path, 'r') as f:
        for line in f:
            line = line.strip().split(delim)
            line = [float(i) for i in line]
            data.append(line)
    return np.asarray(data)

In [3]:
def poly_fit(traj, traj_len, threshold):
    """
    Input:
    - traj: Numpy array of shape (2, traj_len)
    - traj_len: Len of trajectory
    - threshold: Minimum error to be considered for non-linear traj
    Output:
    - int: 1 -> Non Linear 0-> Linear
    """
    t = np.linspace(0, traj_len - 1, traj_len)
    res_x = np.polyfit(t, traj[0, -traj_len:], 2, full=True)[1]
    res_y = np.polyfit(t, traj[1, -traj_len:], 2, full=True)[1]
    if res_x + res_y >= threshold:
        return 1.0
    else:
        return 0.0

In [4]:
num_peds_in_seq = []
seq_list = []
loss_mask_list = []
non_linear_ped = []
frame_id = []
path = 'D:/AmirKabir/tez/eighen trajectory/nuscenes_test_cpu/nuscenes_test_cpu/datasets/eth/train/biwi_hotel_train.txt'
try:
    data = read_file(path)
except:
    data = read_file(path, delim = 'space')
        
frames = np.unique(data[:, 0]).tolist()
frame_data = []
for frame in frames:
    frame_data.append(data[frame == data[:, 0], :])

In [35]:
seq_len = 20
obs_len = 8
skip = 1
pred_len = 12
threshold=0.02
min_ped=1
num_sequences = int(math.ceil((len(frames) - seq_len + 1) / skip))
num_peds_in_seq = []
seq_list = []
loss_mask_list = []
non_linear_ped = []
frame_id = []
pedestrain_id = []
for idx in range(0, num_sequences * skip + 1, skip):
    curr_seq_data = np.concatenate(frame_data[idx:idx + seq_len], axis=0)
    peds_in_curr_seq = np.unique(curr_seq_data[:, 1])
    frame_range = np.unique(curr_seq_data[:, 0])
    curr_seq = np.zeros((len(peds_in_curr_seq), 2, seq_len))
    curr_loss_mask = np.zeros((len(peds_in_curr_seq), seq_len))
    num_peds_considered = 0
    _non_linear_ped = []
    ped_list = []
    for _, ped_id in enumerate(peds_in_curr_seq):
        curr_ped_seq = curr_seq_data[curr_seq_data[:, 1] == ped_id, :]
        curr_ped_seq = np.around(curr_ped_seq, decimals=4)
        pad_front = frames.index(curr_ped_seq[0, 0]) - idx
        pad_end = frames.index(curr_ped_seq[-1, 0]) - idx + 1
        if pad_end - pad_front != seq_len:
            continue
        curr_ped_seq = np.transpose(curr_ped_seq[:, 2:])
        curr_ped_seq = curr_ped_seq
        _idx = num_peds_considered
        curr_seq[_idx, :, pad_front:pad_end] = curr_ped_seq
        # Linear vs Non-Linear Trajectory
        _non_linear_ped.append(poly_fit(curr_ped_seq, pred_len, threshold))
        curr_loss_mask[_idx, pad_front:pad_end] = 1
        num_peds_considered += 1
        ped_list.append(ped_id)


    if num_peds_considered > min_ped:
        non_linear_ped += _non_linear_ped
        num_peds_in_seq.append(num_peds_considered)
        loss_mask_list.append(curr_loss_mask[:num_peds_considered])
        seq_list.append(curr_seq[:num_peds_considered])
        frame_id.append(frame_range)
        pedestrain_id.append(ped_list)

In [36]:
num_seq = len(seq_list)
seq_list = np.concatenate(seq_list, axis=0)
loss_mask_list = np.concatenate(loss_mask_list, axis=0)
frame_lists = [(row.min() , row.max())  for row in frame_id]
non_linear_ped = np.asarray(non_linear_ped)
num_peds_in_seq = np.array(num_peds_in_seq)
pedestrain_id = np.concatenate(pedestrain_id , axis = 0)
# Convert numpy -> Torch T
obs_traj = torch.from_numpy(seq_list[:, :, :obs_len]).type(torch.float).permute(0, 2, 1)  # NTC
pred_traj = torch.from_numpy(seq_list[:, :, obs_len:]).type(torch.float).permute(0, 2, 1)  # NTC
loss_mask = torch.from_numpy(loss_mask_list).type(torch.float)
non_linear_ped = torch.from_numpy(non_linear_ped).type(torch.float)
cum_start_idx = [0] + np.cumsum(num_peds_in_seq).tolist()
seq_start_end = [(start, end) for start, end in zip(cum_start_idx, cum_start_idx[1:])]

In [45]:
num_peds_in_seq

array([3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3,
       3, 3, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5,
       5, 5, 5, 3, 4, 4, 4, 4, 3, 3, 4, 4, 4, 4, 4, 3, 3, 3, 3, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 3, 4, 5, 2, 2, 2, 3, 2, 2, 2, 5, 5, 5, 3, 3,
       5, 5, 5, 4, 5, 3, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 3, 3, 4, 4, 4, 4, 4, 4, 3, 3,
       2, 2, 2, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 2,
       2, 2, 2, 2, 4, 4, 4, 4, 4, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 6, 6, 6, 6, 8, 7, 5, 5, 5])

In [15]:
obs_traj_test.shape

torch.Size([3, 8, 2])

In [9]:
import os
data_dir = "D:/AmirKabir/tez/eighen trajectory/project_git/project/project_gpu/datasets/eth/train/"
all_file = os.listdir(data_dir)
all_file = [os.path.join(data_dir, _path) for _path in all_file]

In [10]:
all_file

['D:/AmirKabir/tez/eighen trajectory/project_git/project/project_gpu/datasets/eth/train/biwi_hotel_train.txt',
 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_gpu/datasets/eth/train/biwi_hotel_train_frames',
 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_gpu/datasets/eth/train/crowds_zara01.txt',
 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_gpu/datasets/eth/train/crowds_zara01_frames',
 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_gpu/datasets/eth/train/crowds_zara02.txt',
 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_gpu/datasets/eth/train/crowds_zara02_frames',
 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_gpu/datasets/eth/train/students003.txt',
 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_gpu/datasets/eth/train/students003_frames']

In [3]:
path = 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_gpu/datasets/eth/train/crowds_zara01.txt'
D:\AmirKabir\tez\eighen trajectory\project_git\project\project_gpu\datasets\hotel\train
splitted_path = path.split('/')
address_until_datasets = '/'.join(splitted_path[:8])
videos_name = splitted_path[-1][:-4] + '_frames'
frame_path = address_until_datasets + '/' + videos_name
frame_path

'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_gpu/datasets/crowds_zara01_frames'

In [4]:
import os
import math
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data.sampler import Sampler
from torch.utils.data.dataloader import DataLoader
from torchvision.transforms import transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def get_dataloader(data_dir, phase, obs_len, pred_len, batch_size):
    r"""Get dataloader for a specific phase

    Args:
        data_dir (str): path to the dataset directory
        phase (str): phase of the data, one of 'train', 'val', 'test'
        obs_len (int): length of observed trajectory
        pred_len (int): length of predicted trajectory
        batch_size (int): batch size

    Returns:
        loader_phase (torch.utils.data.DataLoader): dataloader for the specific phase
    """

    assert phase in ['train', 'val', 'test']
    
    
    data_set = data_dir + '/' + phase + '/'
    shuffle = True if phase == 'train' else False
    drop_last = True if phase == 'train' else False 
    dataset_phase = TrajectoryDataset(data_set, obs_len=obs_len, pred_len=pred_len)
    sampler_phase = None
    if batch_size > 1:
        sampler_phase = TrajBatchSampler(dataset_phase, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    loader_phase = DataLoader(dataset_phase, batch_sampler=sampler_phase ,collate_fn=traj_collate_fn, pin_memory=True)
    return loader_phase


def traj_collate_fn(data):
    r"""Collate function for the dataloader

    Args:
        data (list): list of tuples of (obs_seq, pred_seq, non_linear_ped, loss_mask, seq_start_end)

    Returns:
        obs_seq_list (torch.Tensor): (num_ped, obs_len, 2)
        pred_seq_list (torch.Tensor): (num_ped, pred_len, 2)
        non_linear_ped_list (torch.Tensor): (num_ped,)
        loss_mask_list (torch.Tensor): (num_ped, obs_len + pred_len)
        scene_mask (torch.Tensor): (num_ped, num_ped)
        seq_start_end (torch.Tensor): (num_ped, 2)
        frame_list_tensor
    """

    obs_seq_list, pred_seq_list, non_linear_ped_list, loss_mask_list, _,frame_list_tensor= zip(*data)

    _len = [len(seq) for seq in obs_seq_list]
    cum_start_idx = [0] + np.cumsum(_len).tolist()
    seq_start_end = [[start, end] for start, end in zip(cum_start_idx, cum_start_idx[1:])]
    seq_start_end = torch.LongTensor(seq_start_end)
    scene_mask = torch.zeros(sum(_len), sum(_len), dtype=torch.bool)
    for idx, (start, end) in enumerate(seq_start_end):
        scene_mask[start:end, start:end] = 1

    out = [torch.cat(obs_seq_list, dim=0), torch.cat(pred_seq_list, dim=0),
           torch.cat(non_linear_ped_list, dim=0), torch.cat(loss_mask_list, dim=0), scene_mask, seq_start_end,frame_list_tensor]
    return tuple(out)


class TrajBatchSampler(Sampler):
    r"""Samples batched elements by yielding a mini-batch of indices.
    Args:
        data_source (Dataset): dataset to sample from
        batch_size (int): Size of mini-batch.
        shuffle (bool, optional): set to ``True`` to have the data reshuffled
            at every epoch (default: ``False``).
        drop_last (bool): If ``True``, the sampler will drop the last batch if
            its size would be less than ``batch_size``
        generator (Generator): Generator used in sampling.
    """

    def __init__(self, data_source, batch_size=64, shuffle=False, drop_last=False, generator=None):
        self.data_source = data_source
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.drop_last = drop_last
        self.generator = generator

    def __iter__(self):
        assert len(self.data_source) == len(self.data_source.num_peds_in_seq)

        if self.shuffle:
            if self.generator is None:
                generator = torch.Generator()
                generator.manual_seed(int(torch.empty((), dtype=torch.int64).random_().item()))
            else:
                generator = self.generator
            indices = torch.randperm(len(self.data_source), generator=generator).tolist()
        else:
            indices = list(range(len(self.data_source)))
        num_peds_indices = self.data_source.num_peds_in_seq[indices]

        batch = []
        total_num_peds = 0
        for idx, num_peds in zip(indices, num_peds_indices):
            batch.append(idx)
            total_num_peds += num_peds
            if total_num_peds >= self.batch_size:
                yield batch
                batch = []
                total_num_peds = 0
        if len(batch) > 0 and not self.drop_last:
            yield batch

    def __len__(self):
        # Approximated number of batches.
        # The order of trajectories can be shuffled, so this number can vary from run to run.
        if self.drop_last:
            return sum(self.data_source.num_peds_in_seq) // self.batch_size
        else:
            return (sum(self.data_source.num_peds_in_seq) + self.batch_size - 1) // self.batch_size


def read_file(_path, delim='\t'):
    data = []
    if delim == 'tab':
        delim = '\t'
    elif delim == 'space':
        delim = ' '
    with open(_path, 'r') as f:
        for line in f:
            line = line.strip().split(delim)
            line = [float(i) for i in line]
            data.append(line)
    return np.asarray(data)


def poly_fit(traj, traj_len, threshold):
    """
    Input:
    - traj: Numpy array of shape (2, traj_len)
    - traj_len: Len of trajectory
    - threshold: Minimum error to be considered for non-linear traj
    Output:
    - int: 1 -> Non Linear 0-> Linear
    """
    t = np.linspace(0, traj_len - 1, traj_len)
    res_x = np.polyfit(t, traj[0, -traj_len:], 2, full=True)[1]
    res_y = np.polyfit(t, traj[1, -traj_len:], 2, full=True)[1]
    if res_x + res_y >= threshold:
        return 1.0
    else:
        return 0.0

def prepare_image(frame):
    transform = transforms.Compose([
                      transforms.ToPILImage(),
                      transforms.Resize(300),
                      transforms.ToTensor()])
    transformed_image = transform(frame)
    transformed_image = transformed_image.unsqueeze(0)
    return transformed_image.to(device)



class TrajectoryDataset(Dataset):
    """Dataloder for the Trajectory datasets"""

    def __init__(self, data_dir, obs_len=8, pred_len=12, skip=1, threshold=0.02, min_ped=1, delim='\t'):
        """
        Args:
        - data_dir: Directory containing dataset files in the format <frame_id> <ped_id> <x> <y>
        - obs_len: Number of time-steps in input trajectories
        - pred_len: Number of time-steps in output trajectories
        - skip: Number of frames to skip while making the dataset
        - threshold: Minimum error to be considered for non-linear traj when using a linear predictor
        - min_ped: Minimum number of pedestrians that should be in a sequence
        - delim: Delimiter in the dataset files
        """
        super(TrajectoryDataset, self).__init__()

        self.data_dir = data_dir
        self.obs_len = obs_len
        self.pred_len = pred_len
        self.skip = skip
        self.seq_len = self.obs_len + self.pred_len
        self.delim = delim

        all_file = os.listdir(self.data_dir)
        all_file = [os.path.join(self.data_dir, _path) for _path in all_file]
        num_peds_in_seq = []
        seq_list = []
        frame_list = []
        loss_mask_list = []
        non_linear_ped = []
        pedestrain_id = []#new add
        text_format = 'txt'

        for path in all_file:
            splitted_path = path.split('/')
            address_until_datasets = '/'.join(splitted_path[:8])
            videos_name = splitted_path[-1][:-4] + '_frames'
            frame_path = address_until_datasets + '/' + videos_name

            try:
                data = read_file(path, delim)
                video_path = frame_path
            except:
                data = read_file(path, delim = 'space')    
                video_path = frame_path
                
            frame_id = [] #new add    
            frames = np.unique(data[:, 0]).tolist()
            frame_data = []
            for frame in frames:
                frame_data.append(data[frame == data[:, 0], :])
            num_sequences = int(math.ceil((len(frames) - self.seq_len + 1) / skip))

            for idx in range(0, num_sequences * self.skip + 1, skip):
                curr_seq_data = np.concatenate(frame_data[idx:idx + self.seq_len], axis=0)
                frame_range = np.unique(curr_seq_data[:, 0])       #new add
                peds_in_curr_seq = np.unique(curr_seq_data[:, 1])
                curr_seq = np.zeros((len(peds_in_curr_seq), 2, self.seq_len))
                curr_loss_mask = np.zeros((len(peds_in_curr_seq), self.seq_len))
                num_peds_considered = 0
                _non_linear_ped = []
                ped_list = [] #new add
                for _, ped_id in enumerate(peds_in_curr_seq):
                    curr_ped_seq = curr_seq_data[curr_seq_data[:, 1] == ped_id, :]
                    curr_ped_seq = np.around(curr_ped_seq, decimals=4)
                    pad_front = frames.index(curr_ped_seq[0, 0]) - idx
                    pad_end = frames.index(curr_ped_seq[-1, 0]) - idx + 1
                    if pad_end - pad_front != self.seq_len:
                        continue
                    curr_ped_seq = np.transpose(curr_ped_seq[:, 2:])
                    curr_ped_seq = curr_ped_seq
                    _idx = num_peds_considered
                    curr_seq[_idx, :, pad_front:pad_end] = curr_ped_seq
                    # Linear vs Non-Linear Trajectory
                    _non_linear_ped.append(poly_fit(curr_ped_seq, pred_len, threshold))
                    curr_loss_mask[_idx, pad_front:pad_end] = 1
                    num_peds_considered += 1
                    ped_list.append(ped_id) #new add

                if num_peds_considered > min_ped:
                    non_linear_ped += _non_linear_ped
                    num_peds_in_seq.append(num_peds_considered)
                    loss_mask_list.append(curr_loss_mask[:num_peds_considered])
                    seq_list.append(curr_seq[:num_peds_considered])
                    frame_path = [video_path +'/' +'frame_' + str(int(number)) + '.jpg' for number in frame_range][:8]
                    temp = [plt.imread(path) for path in frame_path]
                    frame_list.append(temp)
                    frame_id.append(frame_range) #new add
                    pedestrain_id.append(ped_list) #new add
                    
            
        self.num_seq = len(seq_list)
        seq_list = np.concatenate(seq_list, axis=0)
        loss_mask_list = np.concatenate(loss_mask_list, axis=0)
        non_linear_ped = np.asarray(non_linear_ped)
         #new add
        self.pedestrain_id = np.concatenate(pedestrain_id , axis = 0) #new add
        self.num_peds_in_seq = np.array(num_peds_in_seq) 

        # Convert numpy -> Torch Tensor
        self.frame_list = frame_list
        self.obs_traj = torch.from_numpy(seq_list[:, :, :self.obs_len]).type(torch.float).permute(0, 2, 1)  # NTC
        self.pred_traj = torch.from_numpy(seq_list[:, :, self.obs_len:]).type(torch.float).permute(0, 2, 1)  # NTC
        self.loss_mask = torch.from_numpy(loss_mask_list).type(torch.float)
        self.non_linear_ped = torch.from_numpy(non_linear_ped).type(torch.float)
        cum_start_idx = [0] + np.cumsum(num_peds_in_seq).tolist()
        self.seq_start_end = [(start, end) for start, end in zip(cum_start_idx, cum_start_idx[1:])]

    def __len__(self):
        return self.num_seq


    def __getitem__(self, index):
        start, end = self.seq_start_end[index]
        out = [self.obs_traj[start:end], self.pred_traj[start:end],
               self.non_linear_ped[start:end], self.loss_mask[start:end], [[0, end - start]], self.frame_list[index]]
        return out 


In [6]:
dataset_dir = 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_gpu/datasets/eth/'
loader_train = get_dataloader(dataset_dir, 'train', 8, 12 ,batch_size=1)

In [13]:
path = './datasets/hotel//train/biwi_eth.txt'
splitted_path = path.split('/')
address_until_datasets = '/'.join(splitted_path[:2])
videos_name = splitted_path[-1][:-4] + '_frames'
frame_path = address_until_datasets + '/' + videos_name

In [15]:
frame_path

'./datasets/biwi_eth_frames'